<h1 align="Center">AF SDK and Python</h1>
<table>
    <tr>
        <td>
            <img src="https://www.python.org/static/community_logos/python-logo-master-v3-TM.png" title="Python Logo" width=350 height=350/>
        </td>
        <td>
            <img src="https://ardi.optrix.com.au/wp-content/uploads/2019/06/OSIsoft_logo.png" title="OSISoft Logo" width=350 height=350/>
        </td>
    </tr>
</table>
<br>
The PI Asset Framework SDK is an invaluable tool to an engineer wanting to take a deeper dive into the workings of PI. By understanding the AF SDK, you will refine your knowledge of how both server and client applications function. Combining this with the limitless use cases of Python opens an opportunity for quickly exploring data, configurations, and errors within the PI System. This notebook will walk through the first steps of setting up a python script to access the PI system from scratch. This obviously will require the AF SDK DLL. You can easily acquire this by install PI System Explorer, which relies of the AF SDK for nearly all of it's functionality.
<h3>Installing required PI Software</h3>
You can find the PI System Explorer executable from OSISoft's downloads website.
Additionally, you will need to have access to a <b>PI Data Archive server</b>.
<h3>Installing required python packages</h3>
The only package that is not installed with python is pythonnet. This package allows us to import the AF SDK DLL.
<blockquote>conda install -c conda-forge pythonnet</blockquote>
    OR if you are not using Anaconda3
<blockquote>pip3 install pythonnet</blockquote>

In [13]:
# pythonnet is imported as "clr"
# this package allows you to import dotNet DLL's and run them within your python environment
import clr
import os
import sys

In [14]:
# Get the installation directory from the environment variable or fall back to the Windows
# default installation path
PIAF_SDK = os.getenv('PIHOME', 'C:\\Program Files\\PIPC')
PIAF_SDK += '\\AF\\PublicAssemblies\\4.0\\'

# check if the AF SDK default directories exist
if not os.path.isdir(PIAF_SDK):
    raise ImportError('PIAF SDK not found in {}, check installation'.format(PIAF_SDK))
    
# append them to the PATH environment variable
sys.path.append(PIAF_SDK)

# add the AF SDK DLL 
clr.AddReference('OSIsoft.AFSDK')

# import the AF SDK
from OSIsoft import AF

print(AF)

<module 'OSIsoft.AF'>


<h3>That's it!</h3>
If the above code ran, you have successfully imported the AF SDK into python.
Let's try to access our PI system. 

<hr>
<h2>Your First Python AF SDK Example</h2>
<hr>
As our first exercise, let's try the following:<br><h3><ol><li>Connect to the Data Archive</li><li>Search for a PI Point</li><li> Get the snapshot value of the PI Point</li></ol></h3>

For reference, the AF SDK has <a href="https://techsupport.osisoft.com/Documentation/PI-AF-SDK/Html/T_OSIsoft_AF_PI_PIServer.htm">online documentation</a>. You can follow along here to discover the properties and methods of all the objects within the AF SDK. We will start by instantiating a <b>PIServers</b> object, and then call the <b>GetPIServers</b> method.



In [15]:
AF.PI.PIServers.GetPIServers()

If you run the above code, you will be returned something like
<blockquote>&lt;OSIsoft.AF.PI.PIServers at 0x1e1c6044fc8&gt;</blockquote>

What the heck? I thought it would return a list of PI servers! Technically, it has returned them as a ".Net object".. sort of! Almost all of the AF SDK will return these objects instead of python friendly objects.

So, how do I get it into pythonic objects? You will need to <em><b>cast</b></em> the object. Essentially, force python to interpret it as a certain data type.

In [16]:
get_piservers_obj = AF.PI.PIServers.GetPIServers()
print(list(get_piservers_obj)) 

[<OSIsoft.AF.PI.PIServer object at 0x000001D22BFE8FD0>, <OSIsoft.AF.PI.PIServer object at 0x000001D2137D0898>, <OSIsoft.AF.PI.PIServer object at 0x000001D213580748>, <OSIsoft.AF.PI.PIServer object at 0x000001D22BF99470>, <OSIsoft.AF.PI.PIServer object at 0x000001D22BF99240>, <OSIsoft.AF.PI.PIServer object at 0x000001D22BFD6978>, <OSIsoft.AF.PI.PIServer object at 0x000001D22BFD6940>, <OSIsoft.AF.PI.PIServer object at 0x000001D22BFD64A8>, <OSIsoft.AF.PI.PIServer object at 0x000001D22BFD6828>]


Wait.. I casted the object to a list, and all it did was return more object(s) in a list!
<blockquote>[&lt;OSIsoft.AF.PI.PIServer object at 0x000001E1C607C448&gt;]</blockquote>
Why didn't it work? Python does not understand the "metadata" of the object, it doesn't understand it is an iterable object that can be casted into a list of objects represented by strings. <b>This is the one of the most challenging parts to using the AF SDK in python.</b> To cast iterables, you will have to "unwrap" them so speak, item by item. Furthermore, every python object has a __repr__ string that informs the interpreter <em>how</em> to print an object. The AF SDK objects do dont contain this, so we must specify the <b>PIServer property</b> which we would like to print. In this case we want the <b>Name Property</b>. If you are already familiar with .NET objects, this will be more intuitive.

In [2]:
# loop through each item in the list, and print the ".Name" of that item
print([item.Name for item in get_piservers_obj])

['my-piserver', 'my-other-pi-server']


Finally, we were able to print out the name of the data archive(s)! Because I only have one data archive, we can assume it is the zeroeth index of the GetPIServers return object.
<br><b>Note: If you have more than one data archive, you will need to change the index below from 0 to the respective index.</b>

In [ ]:
# get first item in GetPIServers object
piserver = AF.PI.PIServers.GetPIServers()[0]

# print data archive name
print(piserver.Name)

<hr>
<h3> 1. Connect to Data Archive</h3>
Now we will attempt to connect. But wait.. I don't know what function to call to make it connect.
We can refer to the AF SDK documentation, navigating the the <a href="https://techsupport.osisoft.com/Documentation/PI-AF-SDK/html/T_OSIsoft_AF_PI_PIServer.htm">PIServer Object page</a>. We can scroll down the the <b>Methods</b> section and we will find our connection methods.<br><img src="./img/piserver_connect.png"></img><br> Wait.. why are there so many, which do I choose? Most methods in the AF SDK will have many parameter set choices, and their use cases each differ slightly. For this case, we will use the first method, in which we just need to pass a boolean value of True.

In [ ]:
# print out our data archive server name
print("Using PI Data Archive: {}".format(piserver.Name))

# Connect to data archive
piserver.Connect(True)
print(piserver.ConnectionInfo)

<hr>
<h3>2. Search for a PI Point</h3>Now that we have verified we can connect to our data archive, we will search for a PI Point.<br>To do this, we actually will instantiate a <a herf="https://techsupport.osisoft.com/Documentation/PI-AF-SDK/html/T_OSIsoft_AF_PI_PIPoint.htm">PIPoint</a> AF SDK object, and invoke the <b>FindPIPoint</b> method.

In [ ]:
# search for pi point on the pi data archive
pipoint = AF.PI.PIPoint.FindPIPoint(piserver, "sinusoid")
print("Found PI Point: {}".format(pipoint.Name))

Excellent! We now have found a PI Point on our data archive. We now are able to access the objects methods.

<hr>
<h3>3. Get the snapshot value of the PI Point</h3>
For the final step in the exercise, let's return a value using the <b>CurrentValue</b> method.

In [ ]:
# get current value
val = pipoint.CurrentValue()
print("The current value of {} is {}".format(pipoint.Name, val))

<b>Whew!</b> It may have seemed like a lot of work to print a point value.. I mean you could have opened SMT and got here much quicker. However, you are no longer bound by the GUI, and you have access to all AF SDK objects in a programmatic manner.
<hr>
That being said, the AF SDK and python are <b><em>very powerful</em></b> tools. You must use them carefully. Many of the built-in checks that come with PI client tools can be bypassed, for better or for worst. Always test your code in a test environment first, and only resort to python code when necessary.
<img src="https://media.giphy.com/media/MCZ39lz83o5lC/giphy.gif" width="480" height="257"></img>